Please Fill in your name below:

In [ ]:
NAME = "Mohammed Ali Ali"

---

# Deep Learning
# Assignment 3: Convolutional Neural Networks

**Please follow the following instructions for your answers to be graded correctly**:
- Inside the functions, make sure to write your code between the comments "# YOUR CODE STARTS HERE " and "# YOUR CODE ENDS HERE ". **Do not** edit any code outside these comments inside the functions.
- You're allowed to edit the cells that don't contain function definition.
- You will encounter some locked (read-only) empty cells. Leave those as they are, you're not allowed to edit them.
- Before you submit your assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

## Part 1: CNN

In [ ]:
import numpy
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import utils
import matplotlib.pyplot as plt

### Data loading and preprocessing

In [ ]:
def load_cifar():
    '''
    Returns:
    X_train, y_train, X_test, y_test --> images should be normalized to the range 0-1. Labels should be one-hot encoded.
    '''

    # YOUR CODE STARTS HERE
    X
    # YOUR CODE ENDS HERE

    return X_train, y_train, X_test, y_test

### CNN model

In [ ]:
def CNN (learning_rate):
    '''
    Implement a 3 layer CNN with filter size 3x3, where num of filters 32, 64 and 128 added to the first, second and third layer respectively.
    Add a maxpool layer with pooling size (2,2) after each convolutional layer.
    The CNN is followed by a two dense layers where the num of nodes are 128 and 10 in the first and second dense layers (Use softmax at the last layer).
    Use SGD optimization function where the loss function is employing the categorical cross entropy to classify the images.

    Arguments:
    learning_rate

    Returns:
    model: the compiled cnn model
    '''

    # YOUR CODE STARTS HERE
    m
    # YOUR CODE ENDS HERE

    return model

In [ ]:
# Fit your CNN model using test data as your validation data during training.
# Plot the training accuracy and validation accuracy (add legend to the plot)

# YOUR CODE STARTS HERE

# YOUR CODE ENDS HERE

### DNN model

In [ ]:
def DNN(learning_rate):
    '''
    Implement a simple dense neural networks with 5 hidden layers of 512 rectified linear units each. Add a final output layer with softmax activation.

    Arguments:
    learning_rate

    Returns:
    model: the compiled model
    '''

    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    return model

In [ ]:
# Fit your DNN model using test data as your validation data during training.
# Plot the training accuracy and validation accuracy (add legend to the plot)

# YOUR CODE STARTS HERE

# YOUR CODE ENDS HERE

### Question: Compare between both CNN and DNN models in terms of the training curves and number of parameters.

YOUR ANSWER HERE

### Dropout

In [ ]:
def CNN_dropout(dropout_rate, learning_rate):
    '''
    Implement the same previous CNN architecture with adding dropout layers.

    Arguments:
    - dropout_rate
    - learning_rate

    returns:
    model: the compiled model
    '''
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    return model

In [ ]:
# Fit your modified CNN model (CNN with dropout) using test data as your validation data during training.
# Plot the training accuracy and validation accuracy (add legend to the plot)


# YOUR CODE STARTS HERE

# YOUR CODE ENDS HERE

In [ ]:
# Repeat using a different dropout rate

# YOUR CODE STARTS HERE

# YOUR CODE ENDS HERE

### Comment on the results using different dropout rates and compare with the previous models.

YOUR ANSWER HERE

## Part 2: UNet

You'll be building your own U-Net.

### 1 - Packages

Run the cell below to import all the libraries you'll need:

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate

### 2 - Load and Split the Data

- Download the data from Lyft Semantic Segmentation Challenge from Kaggle.

    - Link:https://www.kaggle.com/datasets/kumaresanmanickavelu/lyft-udacity-challenge

- Use the **DataA**  folder only.

- You can run on Colab.
    - upload data to drive
    - OR fetch the data from Kaggle to Colab directly (easier)
    
- OR You can run on Kaggle directly (EASIER)

In [ ]:
import os
import numpy as np
import imageio

import matplotlib.pyplot as plt
%matplotlib inline

path = #........

image_path = os.path.join(path, 'CameraRGB/')
mask_path = os.path.join(path, 'CameraSeg/')
image_list = os.listdir(image_path)
mask_list = os.listdir(mask_path)
image_list = [image_path+i for i in image_list]
mask_list = [mask_path+i for i in mask_list]

In [ ]:
len(image_list)

#### Check out the some of the unmasked and masked images from the dataset:

In [ ]:
N = 3
img = imageio.imread(image_list[N])
mask = imageio.imread(mask_list[N])

fig, arr = plt.subplots(1, 2, figsize=(14, 10))
arr[0].imshow(img)
arr[0].set_title('Image')
arr[1].imshow(mask[:, :, 0])
arr[1].set_title('Segmentation')

#### 2.1 - Split Your Dataset into Unmasked and Masked Images

In [ ]:
image_list_ds = tf.data.Dataset.list_files(image_list, shuffle=False)
mask_list_ds = tf.data.Dataset.list_files(mask_list, shuffle=False)

for path in zip(image_list_ds.take(3), mask_list_ds.take(3)):
    print(path)

In [ ]:
image_filenames = tf.constant(image_list)
masks_filenames = tf.constant(mask_list)

dataset = tf.data.Dataset.from_tensor_slices((image_filenames, masks_filenames))

for image, mask in dataset.take(1):
    print(image)
    print(mask)

#### 2.2 - Preprocess Your Data

In [ ]:
def process_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)
    return img, mask

def preprocess(image, mask):
    input_image = tf.image.resize(image, (96, 128), method='nearest')
    input_mask = tf.image.resize(mask, (96, 128), method='nearest')

    input_image = input_image / 255.

    return input_image, input_mask

image_ds = dataset.map(process_path)
processed_image_ds = image_ds.map(preprocess)

### 3 - U-Net

#### 3.1 - Model Details

<a href="https://ibb.co/8bfBpz5"><img src="https://i.ibb.co/QkTcR8N/unet.png" alt="unet" border="0"></a><br /><a target='_blank' href='https://emoticoncentral.com/category/classical-building'></a><br />
<caption><center> <u><b> Figure 2 </u></b>: U-Net Architecture<br> </center></caption>

**Contracting path** (Encoder containing downsampling steps):

Images are first fed through several convolutional layers which reduce height and width, while growing the number of channels.

The contracting path follows a regular CNN architecture, with convolutional layers, their activations, and pooling layers to downsample the image and extract its features. In detail, it consists of the repeated application of two 3 x 3 unpadded convolutions, each followed by a rectified linear unit (ReLU) and a 2 x 2 max pooling operation with stride 2 for downsampling. At each downsampling step, the number of feature channels is doubled.

**Crop function**: This step crops the image from the contracting path and concatenates it to the current image on the expanding path to create a skip connection.

**Expanding path** (Decoder containing upsampling steps):

The expanding path performs the opposite operation of the contracting path, growing the image back to its original size, while shrinking the channels gradually.

In detail, each step in the expanding path upsamples the feature map, followed by a 2 x 2 convolution (the transposed convolution). This transposed convolution halves the number of feature channels, while growing the height and width of the image.

Next is a concatenation with the correspondingly cropped feature map from the contracting path, and two 3 x 3 convolutions, each followed by a ReLU. You need to perform cropping to handle the loss of border pixels in every convolution.

**Final Feature Mapping Block**: In the final layer, a 1x1 convolution is used to map each 64-component feature vector to the desired number of classes. The channel dimensions from the previous layer correspond to the number of filters used, so when you use 1x1 convolutions, you can transform that dimension by choosing an appropriate number of 1x1 filters. When this idea is applied to the last layer, you can reduce the channel dimensions to have one layer per class.

The U-Net network has 23 convolutional layers in total.

#### 3.2 - Encoder (Downsampling Block)


<a href="https://ibb.co/BGGWBGc"><img src="https://i.ibb.co/XDD0tDj/encoder.png" alt="encoder" border="0"></a>
<caption><center> <u><b>Figure 3</u></b>: The U-Net Encoder up close <br> </center></caption>

The encoder is a stack of various conv_blocks:

Each `conv_block()` is composed of 2 **Conv2D** layers  with ReLU activations. We will apply **Dropout**, and **MaxPooling2D** to some conv_blocks, as you will verify in the following sections, specifically to the last two blocks of the downsampling.

The function will  return two tensors:
- `next_layer`: That will go into the next block.
- `skip_connection`: That will go into the corresponding decoding block.

**Note**: If `max_pooling=True`, the `next_layer` will be the output of the MaxPooling2D layer, but the `skip_connection` will be the output of the previously applied layer(Conv2D or Dropout, depending on the case). Else, both results will be identical.  

#### Exercise 1 - conv_block

Implement `conv_block(...)`. Here are the instructions for each step in the `conv_block`, or contracting block:

* Add 2 **Conv2D** layers with `n_filters` filters with `kernel_size` set to 3, `kernel_initializer` set to ['he_normal'](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal), `padding` set to 'same' and 'relu' activation.
* if `dropout_prob` > 0, then add a Dropout layer with parameter `dropout_prob`
* If `max_pooling` is set to True, then add a MaxPooling2D layer with 2x2 pool size

In [ ]:
def conv_block(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):
    """
    Convolutional downsampling block

    Arguments:
        inputs -- Input tensor
        n_filters -- Number of filters for the convolutional layers
        dropout_prob -- Dropout probability
        max_pooling -- Use MaxPooling2D to reduce the spatial dimensions of the output volume
    Returns:
        next_layer, skip_connection --  Next layer and skip connection outputs
    """

    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    # if dropout_prob > 0 add a dropout layer, with the variable dropout_prob as parameter
    if dropout_prob > 0:
         # YOUR CODE STARTS HERE

         # YOUR CODE ENDS HERE


    # if max_pooling is True add a MaxPooling2D with 2x2 pool_size
    if max_pooling:
        # YOUR CODE STARTS HERE

        # YOUR CODE ENDS HERE

    else:
        next_layer = conv

    skip_connection = conv

    return next_layer, skip_connection

#### 3.3 - Decoder (Upsampling Block)

The decoder, or upsampling block, upsamples the features back to the original image size. At each upsampling level, you'll take the output of the corresponding encoder block and concatenate it before feeding to the next decoder block.


<a href="https://ibb.co/0rGzrk1"><img src="https://i.ibb.co/VTqkTdb/decoder.png" alt="decoder" border="0"></a>
<caption><center> <u><b>Figure 4</u></b>: The U-Net Decoder up close <br> </center></caption>

There are two new components in the decoder: `up` and `merge`. These are the transpose convolution and the skip connections. In addition, there are two more convolutional layers set to the same parameters as in the encoder.

Here you'll encounter the `Conv2DTranspose` layer, which performs the inverse of the `Conv2D` layer. You can read more about it [here.](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose)


#### Exercise 2 - upsampling_block

Implement `upsampling_block(...)`.

For the function `upsampling_block`:
* Takes the arguments `expansive_input` (which is the input tensor from the previous layer) and `contractive_input` (the input tensor from the previous skip layer)
* The number of filters here is the same as in the downsampling block you completed previously
* Your `Conv2DTranspose` layer will take `n_filters` with shape (3,3) and a stride of (2,2), with padding set to `same`. It's applied to `expansive_input`, or the input tensor from the previous layer.

This block is also where you'll concatenate the outputs from the encoder blocks, creating skip connections.

* Concatenate your Conv2DTranspose layer output to the contractive input, with an `axis` of 3. In general, you can concatenate the tensors in the order that you prefer. But for the grader, it is important that you use `[up, contractive_input]`

For the final component, set the parameters for two Conv2D layers to the same values that you set for the two Conv2D layers in the encoder (ReLU activation, He normal initializer, `same` padding).


In [ ]:
def upsampling_block(expansive_input, contractive_input, n_filters=32):
    """
    Convolutional upsampling block

    Arguments:
        expansive_input -- Input tensor from previous layer
        contractive_input -- Input tensor from previous skip layer
        n_filters -- Number of filters for the convolutional layers
    Returns:
        conv -- Tensor output
    """
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    return conv

#### 3.4 - Build the Model

This is where you'll put it all together, by chaining the encoder, bottleneck, and decoder! You'll need to specify the number of output channels, which for this particular set would be 23. That's because there are **23 possible labels for each pixel in this self-driving car dataset.**

#### Exercise 3 - unet_model

For the function `unet_model`, specify the input shape, number of filters, and number of classes (13 in this case).

For the first half of the model:

* Begin with a conv block that takes the inputs of the model and the number of filters
* Then, chain the first output element of each block to the input of the next convolutional block
* Next, double the number of filters at each step
* Beginning with `conv_block4`, add `dropout` of 0.3
* For the final conv_block, set `dropout` to 0.3 again, and turn off max pooling. (Here `n_filters=n_filters* 16`)

For the second half:
* From here, at each step, use half the number of filters of the previous block
* Use cblock5 as expansive_input and cblock4 as contractive_input, with `n_filters` * 8. This is your bottleneck layer.
* Chain the output of the previous block as expansive_input and the corresponding contractive block output.
* Note that you must use the second element of the contractive block before the max pooling layer.
* `conv9` is a Conv2D layer with ReLU activation, He normal initializer, `same` padding
* Finally, `conv10` is a Conv2D that takes the number of classes as the filter, a kernel size of 1, and "same" padding. The output of `conv10` is the output of your model.

In [ ]:
def unet_model(input_size=(96, 128, 3), n_filters=32, n_classes=13):
    """
    Unet model

    Arguments:
        input_size -- Input shape
        n_filters -- Number of filters for the convolutional layers
        n_classes -- Number of output classes
    Returns:
        model -- tf.keras.Model
    """
    inputs = Input(input_size)
    # Contracting Path (encoding)
    # Add a conv_block with the inputs of the unet_ model and n_filters
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    # Expanding Path (decoding)
    # Add the first upsampling_block.
    # From here,at each step, use half the number of filters of the previous block
    # Use the cblock5[0] as expansive_input and cblock4[1] as contractive_input and n_filters * 8
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

    # Add a Conv2D layer with n_classes filter, kernel size of 1 and a 'same' padding
    # YOUR CODE STARTS HERE

    # YOUR CODE ENDS HERE

    model = tf.keras.Model(inputs=inputs, outputs=conv10)

    return model

#### 3.5 - Set Model Dimensions

In [ ]:
img_height = 96
img_width = 128
num_channels = 3

unet = unet_model((img_height, img_width, num_channels))

#### Check out the model summary below!

In [ ]:
unet.summary()

#### 3.6 - Loss Function

In semantic segmentation, you need as many masks as you have object classes. In the dataset you're using, each pixel in every mask has been assigned a single integer probability that it belongs to a certain class, from 0 to num_classes-1. The correct class is the layer with the higher probability.

This is different from categorical crossentropy, where the labels should be one-hot encoded (just 0s and 1s). Here, you'll use sparse categorical crossentropy as your loss function, to perform pixel-wise multiclass prediction. Sparse categorical crossentropy is more efficient than other loss functions when you're dealing with lots of classes.

In [ ]:
unet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#### 3.7 - Dataset Handling

Below, define a function that allows you to display both an input image, and its ground truth: the true mask. The true mask is what your trained model output is aiming to get as close to as possible.

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
for image, mask in image_ds.take(1):
    sample_image, sample_mask = image, mask
    print(mask.shape)
display([sample_image, sample_mask])

In [ ]:
for image, mask in processed_image_ds.take(1):
    sample_image, sample_mask = image, mask
    print(mask.shape)
display([sample_image, sample_mask])

### 4 - Train the Model

In [ ]:
EPOCHS = 40
BUFFER_SIZE = 500
BATCH_SIZE = 32
processed_image_ds.batch(BATCH_SIZE)
train_dataset = processed_image_ds.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(processed_image_ds.element_spec)
model_history = unet.fit(train_dataset, epochs=EPOCHS)

#### 4.1 - Create Predicted Masks

Now, define a function that uses `tf.argmax` in the axis of the number of classes to return the index with the largest value and merge the prediction into a single image:

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

#### 4.2 - Plot Model Accuracy

Let's see how your model did!

In [ ]:
plt.plot(model_history.history["accuracy"])

#### 4.3 - Show Predictions

Next, check your predicted masks against the true mask and the original input image:

In [ ]:
def show_predictions(dataset=None, num=1):
    """
    Displays the first image of each of the num batches
    """
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = unet.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
             create_mask(unet.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions(train_dataset, 6)